# Tiền xử lý dữ liệu
File này sẽ thực hiện các bước xử lý dữ liệu để chuẩn bị cho quá trình huấn luyện mô hình.

Mục tiêu chính:
1. Kiểm tra dữ liệu và xử lý thiếu giá trị (nếu có).
2. Chuẩn hóa và chuẩn hóa lại dữ liệu, đặc biệt là các cột 'Time' và 'Amount'.
3. Phân tích tương quan giữa các đặc trưng để xác định những đặc trưng quan trọng.
4. Lưu bộ dữ liệu đã xử lý vào thư mục 'data/processed/'.


### 1. Load dữ liệu

#### Thư viện cần thiết

In [ ]:
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#### Đọc dữ liệu

In [ ]:
# Đường dẫn dữ liệu
data_path = "../data/raw/creditcard.csv"
figure_dir = "../reports/figures"

# Đọc dữ liệu
data = pd.read_csv(data_path)

# Thông tin cơ bản về dữ liệu
print("Thông tin dữ liệu:")
data.info()

# Xem 5 dòng đầu tiên
data.head()


### 2. Xử lý thiếu dữ liệu

In [ ]:
# Kiểm tra giá trị thiếu
print("\nKiểm tra giá trị thiếu:")
missing_values = data.isnull().sum()
print(missing_values)

# Xác nhận không có giá trị thiếu
if missing_values.sum() == 0:
    print("Không có giá trị thiếu trong dữ liệu.")


Dữ liệu này không có giá trị thiếu nên không cần thực hiện bước điền giá trị.


### 3. Loại bỏ ngoại lệ (Outlier Detection & Removal)

#### Phát hiện ngoại lệ bằng IQR (Interquartile Range)

In [ ]:
# Tính phân vị 99.5 của Amount trong lớp không gian lận (Class = 0)
threshold = data[data['Class'] == 0]['Amount'].quantile(0.9995)

print(f"Ngưỡng loại bỏ cho Amount (dựa trên phân vị 99.95): {threshold}")

# Loại bỏ các giao dịch có Amount vượt quá ngưỡng trong lớp không gian lận
data_cleaned = data[~((data['Class'] == 0) & (data['Amount'] > threshold))]

# Kết quả trước và sau khi loại bỏ
print(f"Số lượng giao dịch ban đầu: {len(data_cleaned)}")
print(f"Số lượng giao dịch sau khi loại bỏ ngoại lệ: {len(data_cleaned)}")


#### So sánh trước và sau khi loại bỏ ngoại lệ (Visualization)

In [ ]:
plt.figure(figsize=(8,6))
sns.boxplot(x='Class', y='Amount',data = data_cleaned)
plt.title('Phân phối số tiền cho các giao dịch gian lận và không giân lận')
plt.savefig(os.path.join(figure_dir, "8_boxplot_outlier.png"))
plt.show()

## 4. Chuẩn hóa dữ liệu (Normalization vs. Standardization)


Chuẩn hóa cột Amount và Time: Giúp các thuật toán học máy không bị ảnh hưởng bởi sự khác biệt về thang đo.

#### 4.1. Log Transformation

Giải thích: Chuyển đổi Amount để giảm ảnh hưởng của các giá trị cực lớn.

In [ ]:
# Log transform cho cột Amount
data_cleaned['Log_Amount'] = np.log(data_cleaned['Amount'] + 0.0001)

# So sánh trước và sau chuẩn hóa
print("Trước chuẩn hóa:", data_cleaned['Amount'].describe())
print("Sau chuẩn hóa:", data_cleaned['Log_Amount'].describe())


#### 4.2. Standardization

Giải thích: Chuẩn hóa dữ liệu bằng cách đưa về phân phối chuẩn (mean = 0, std = 1).

In [ ]:
from sklearn.preprocessing import StandardScaler

# Standardization cho Amount và Time
scaler = StandardScaler()
data_cleaned['Scaled_Amount'] = scaler.fit_transform(data_cleaned[['Amount']])
data_cleaned['Scaled_Time'] = scaler.fit_transform(data_cleaned[['Time']])


#### 4.3. Normalization (Min-Max)

Giải thích: Chuẩn hóa cột Amount trong khoảng [0, 1].

In [ ]:
from sklearn.preprocessing import MinMaxScaler

minmax_scaler = MinMaxScaler()
data_cleaned['Normalized_Amount'] = minmax_scaler.fit_transform(data_cleaned[['Amount']])


#### Trực quan hoá sau khi chuẩn hoá

In [ ]:
fig , axs = plt.subplots(nrows = 1 , ncols = 4 , figsize = (16,4))

sns.boxplot(x ="Class",y="Amount",data=data_cleaned, ax = axs[0])
axs[0].set_title("Class vs Amount")

sns.boxplot(x ="Class",y="Log_Amount",data=data_cleaned, ax = axs[1])
axs[1].set_title("Class vs Log Amount")

sns.boxplot(x ="Class",y="Scaled_Amount",data=data_cleaned, ax = axs[2])
axs[2].set_title("Class vs Scaled Amount")

sns.boxplot(x ="Class",y="Normalized_Amount",data=data_cleaned, ax = axs[3])
axs[3].set_title("Class vs Min Max Amount")

# fig.suptitle('Amount by Class', fontsize=20)
plt.savefig(os.path.join(figure_dir, "9_class_with_normalization.png"))
plt.show()

### 5. Lưu dữ liệu

In [ ]:
# Lưu dữ liệu đã xử lý
data_cleaned.to_csv('../data/processed/processed_data.csv', index=False)
print("Dữ liệu đã được lưu.")
